In [ ]:
# Required packages
import cv2
import h5py
import numpy as np
import pandas as pd
import glob
import os
import re
from scipy.interpolate import interp1d

In [ ]:
# Load video
video_path = r"C:\Users\irs3th\Desktop\homo_vid_h5\veh_hfd_arena4_1006_re.mp4"
cap = cv2.VideoCapture(video_path)

# Read any example frame
frame_number = 100
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
ret, frame = cap.read()

if ret:
    # Save as an image
    cv2.imwrite('extracted_frame.jpg', frame)
else:
    print("Error extracting frame")

cap.release()

In [ ]:
# Load the SLEAP H5 file
h5_file_path = r"C:\Users\irs3th\Desktop\homo_vid_h5\dan_sd_arena4_1006_re.h5"
with h5py.File(h5_file_path, "r") as f:
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]

# Fill in missing values (code from https://sleap.ai/notebooks/Analysis_examples.html) :)
def fill_missing(locations):
    for i in range(locations.shape[1]):  # Loop through each body part
        for j in range(locations.shape[2]):  # Loop through each coordinate (x or y)
            mask = np.isnan(locations[:, i, j, 0])
            idx = np.where(~mask, np.arange(mask.shape[0]), 0)
            idx = np.maximum.accumulate(idx, axis=0)
            locations[:, i, j, 0] = locations[idx, i, j, 0]
    return locations

# Save as locations
locations = fill_missing(locations)

In [ ]:
SHELTER

In [ ]:
# Draw a box around region of interest (shelter) to get the coordinate location

# Press 'r' to reset the cropping region.
# Press 'c' or 'q' to quit the program and close the window.

refPt = []
cropping = False

def click_and_crop(event, x, y, flags, param):
    global refPt, cropping

    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True

    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        cropping = False

        # draw a rectangle around the region of interest
        cv2.rectangle(image, refPt[0], refPt[1], (0, 255, 0), 2)
        cv2.imshow("image", image)

# Load the image
image = cv2.imread('extracted_frame.jpg')
clone = image.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

# Keep looping until the 'q' or 'c' key is pressed
while True:
    cv2.imshow("image", image)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("r"):
        image = clone.copy()

    elif key == ord("c") or key == ord("q"):
        break

# Close open windows
cv2.destroyAllWindows()

# Print the coordinates
if len(refPt) == 2:
    print(f'shelter coordinates: {refPt}')
    x1, y1 = refPt[0]
    x2, y2 = refPt[1]
    width = abs(x2 - x1)
    height = abs(y2 - y1)
    print(f'shelter width: {width} pixels')
    print(f'shelter height: {height} pixels')

In [ ]:
# Create a dataframe with True (in shelter) or False (not in shelter) for every frame in video

# Define the coordinates from above (top-left and bottom-right)
x1, y1 = 550, 24
x2, y2 = 728, 176

# Index for center 3 location
CENTER3_INDEX = 5
center3_loc = locations[:, CENTER3_INDEX, :, :]

# Create a list to store whether the animal is in the shelter for each frame
in_box = []

for frame in range(center3_loc.shape[0]):
    x, y = center3_loc[frame, 0, 0], center3_loc[frame, 1, 0]
    if x1 <= x <= x2 and y1 <= y <= y2:
        in_box.append(True)
    else:
        in_box.append(False)

# Create a DF with the frame index and whether the animal is in the shelter
df = pd.DataFrame({
    'frame_index': range(center3_loc.shape[0]),
    'shelter': in_box
})

df.insert(0, 'id', '1006') # manually change
df.insert(1, 'group', 'dan_sd') # manually change

# Save to a CSV file
df.to_csv("shelter_dan_sd_1006.csv", index=False) # manually change

# Additional calculations for total time spent in the shelter
frames_in_box = sum(in_box)
fps = 25
time_in_box_seconds = frames_in_box / fps

print(f'Total number of frames spent in the shelter: {frames_in_box}')
print(f'Total time spent in the shelter: {time_in_box_seconds:.2f} seconds')

In [ ]:
FOOD HOPPER

In [ ]:
# Draw a box around region of interest (hopper) to get the coordinate location

# Press 'r' to reset the cropping region.
# Press 'c' or 'q' to quit the program and close the window.

refPt = []
cropping = False

def click_and_crop(event, x, y, flags, param):
    global refPt, cropping

    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True

    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        cropping = False

        # draw a rectangle around the region of interest
        cv2.rectangle(image, refPt[0], refPt[1], (0, 255, 0), 2)
        cv2.imshow("image", image)

# Load the image
image = cv2.imread('extracted_frame.jpg')
clone = image.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

# Keep looping until the 'q' or 'c' key is pressed
while True:
    cv2.imshow("image", image)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("r"):
        image = clone.copy()

    elif key == ord("c") or key == ord("q"):
        break

# Close open windows
cv2.destroyAllWindows()

# Print the coordinates
if len(refPt) == 2:
    print(f'food hopper coordinates: {refPt}')
    x1, y1 = refPt[0]
    x2, y2 = refPt[1]
    width = abs(x2 - x1)
    height = abs(y2 - y1)
    print(f'hopper width: {width} pixels')
    print(f'hopper height: {height} pixels')

In [ ]:
# Create a dataframe with True (by food hopper) or False (not by food hopper) for every frame in video

# Define the coordinates from above (top-left and bottom-right)
x1, y1 = 640, 268
x2, y2 = 776, 414


# Index for nose (since I am interested in sniffing near the hopper)
NOSE_INDEX = 0
nose_loc = locations[:, NOSE_INDEX, :, :]

# Create a list to store whether the animal is near the hopper for each frame
in_box = []

for frame in range(nose_loc.shape[0]):
    x, y = nose_loc[frame, 0, 0], nose_loc[frame, 1, 0]
    if x1 <= x <= x2 and y1 <= y <= y2:
        in_box.append(True)
    else:
        in_box.append(False)


# Create a DataFrame with the frame index and whether the animal is near the hopper
df = pd.DataFrame({
    'frame_index': range(nose_loc.shape[0]),
    'food_hopper': in_box
})

df.insert(0, 'id', '1006') # manually change
df.insert(1, 'group', 'dan_sd') # manually change

# Save to a CSV file
df.to_csv("food_hopper_dan_sd_1006.csv", index=False) # manually change

# Additional calculations for total time spent near the food hopper
frames_in_box = sum(in_box)
fps = 25
time_in_box_seconds = frames_in_box / fps

print(f'Total number of frames spent by the hopper: {frames_in_box}')
print(f'Total time spent by the hopper: {time_in_box_seconds:.2f} seconds')

In [ ]:
WHEEL

In [ ]:
# Draw a box around region of interest (wheel) to get the coordinate location

# Press 'r' to reset the cropping region.
# Press 'c' or 'q' to quit the program and close the window.

refPt = []
cropping = False

def click_and_crop(event, x, y, flags, param):
    global refPt, cropping

    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True

    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        cropping = False

        # draw a rectangle around the region of interest
        cv2.rectangle(image, refPt[0], refPt[1], (0, 255, 0), 2)
        cv2.imshow("image", image)

# Load the image
image = cv2.imread('extracted_frame.jpg')
clone = image.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

# Keep looping until the 'q' or 'c' key is pressed
while True:
    cv2.imshow("image", image)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("r"):
        image = clone.copy()

    elif key == ord("c") or key == ord("q"):
        break

# Close open windows
cv2.destroyAllWindows()

# Print the coordinates
if len(refPt) == 2:
    print(f'wheel coordinates: {refPt}')
    x1, y1 = refPt[0]
    x2, y2 = refPt[1]
    width = abs(x2 - x1)
    height = abs(y2 - y1)
    print(f'wheel width: {width} pixels')
    print(f'wheel height: {height} pixels')

In [ ]:
# Create a dataframe with True (by wheel) or False (not by wheel) for every frame in video

# Define the coordinates from above (top-left and bottom-right)
x1, y1 = 182, 103
x2, y2 = 365, 419


# Index for center 3 location
CENTER3_INDEX = 5
center3_loc = locations[:, CENTER3_INDEX, :, :]

# Create a list to store whether the animal is on the wheel
in_box = []

for frame in range(center3_loc.shape[0]):
    x, y = center3_loc[frame, 0, 0], center3_loc[frame, 1, 0]
    if x1 <= x <= x2 and y1 <= y <= y2:
        in_box.append(True)
    else:
        in_box.append(False)


# Create a DataFrame with the frame index and whether the animal is on the wheel
df = pd.DataFrame({
    'frame_index': range(center3_loc.shape[0]),
    'on_wheel': in_box
})

df.insert(0, 'id', '1006') # manually change
df.insert(1, 'group', 'dan_sd') # manually change

# Save to a CSV file
df.to_csv("wheel_dan_sd_1006.csv", index=False) # manually change

# Additional calculations for total time spent on the wheel
frames_in_box = sum(in_box)
fps = 25
time_in_box_seconds = frames_in_box / fps

print(f'Total number of frames spent on the wheel: {frames_in_box}')
print(f'Total time spent on the wheel: {time_in_box_seconds:.2f} seconds')


In [ ]:
# Read the CSV files
shelter = pd.read_csv('shelter_veh_sd_1006.csv')
food = pd.read_csv('food_hopper_veh_sd_1006.csv')
wheel = pd.read_csv('wheel_veh_sd_1006.csv')

# Merge the DFs on the 'id,' 'group,' and 'frame_index' columns
output1 = pd.merge(shelter, food, on=['id', 'group', 'frame_index'], how='outer')
output1 = pd.merge(output1, wheel, on=['id', 'group','frame_index'], how='outer')

# Save to a CSV file
output1.to_csv("location_veh_sd_1006.csv", index=False)

In [ ]:
output1 # check work